# DES

In [35]:
from Crypto.Cipher import DES
from Crypto.Cipher import DES3
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

In [36]:
def des_encrypt(key, plaintext):
    cipher = DES.new(key, DES.MODE_ECB)
    padded_plaintext = pad(plaintext, DES.block_size)
    ciphertext = cipher.encrypt(padded_plaintext)
    return ciphertext

def des_decrypt(key, ciphertext):
    cipher = DES.new(key, DES.MODE_ECB)
    padded_plaintext = cipher.decrypt(ciphertext)
    plaintext = unpad(padded_plaintext, DES.block_size)
    return plaintext

def des3_encrypt(key, plaintext):
    cipher = DES3.new(key, DES3.MODE_ECB)
    padded_text = pad(plaintext, DES3.block_size)
    return cipher.encrypt(padded_text)

def des3_decrypt(key, ciphertext):
    cipher = DES3.new(key, DES3.MODE_ECB)
    decrypted_padded_text = cipher.decrypt(ciphertext)
    return unpad(decrypted_padded_text, DES3.block_size)

In [55]:
from datetime import datetime

def str_xor(s1, s2):
    bytes1 = bytes.fromhex(s1)
    bytes2 = bytes.fromhex(s2)
    xor_result = bytes(a ^ b for a, b in zip(bytes1, bytes2))
    return xor_result.hex().upper()

def xor_arrays(a, b):
    return bytes([x ^ y for x, y in zip(a, b)])

    
def process_macdata(key, macdata):
    iv = b'\x00' * 8
    des = bytearray(8)
    num_blocks = len(macdata) // 8
    for i in range(num_blocks):
        part = macdata[i * 8:(i + 1) * 8]
        if i == 0:
            part = xor_arrays(part, iv)
            des[:] = des_encrypt(key, part)
        else:
            part = xor_arrays(des, part)
            des[:] = des_encrypt(key, part)
    return bytes(des)

current_time = datetime.now()
DateTime = current_time.strftime("%Y%m%d%H%M%S")
DateTime = "20250123172635"
defaultKey = bytes.fromhex("32D464AC81F1640A687D023BF99E35DF")
posId      = "040900010001"
CardNo = "03104870494120264165"
OnlineSeqNo = "0445"
RandomNo = "A98AA3F7"
factor = CardNo[-16:]
tradeMoney = "00000000"
macdata = tradeMoney + "09" + posId + DateTime + "80" + "0000000000"
macdata = bytes.fromhex(macdata)

In [56]:

xor_result = str_xor(factor, "FFFFFFFFFFFFFFFF")
factor = factor + xor_result
factor = bytes.fromhex(factor)
print("XOR Result:", xor_result)
print("Factor:", factor.hex())

XOR Result: B78FB6BEDFD9BE9A
Factor: 4870494120264165b78fb6bedfd9be9a


In [57]:
loadKey = des3_encrypt(defaultKey, factor)[:16]
print("Load Key:", loadKey.hex())
sessionKey = des3_encrypt(loadKey, bytes.fromhex(RandomNo + OnlineSeqNo + "0001"))[:8]
print("Session Key:", sessionKey.hex())

Load Key: 6e5583005783a27c913e6fb763b959ba
Session Key: b38be96346175ec8


In [58]:
Mac = process_macdata(sessionKey, macdata)[:4].hex()
Mac

'3b94fe09'